In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
import cv2
import numpy as np
import tensorflow as tf

# Set the path to the directory and image name
image_path = "/kaggle/input/ripeness-detection-of-mango/Stage_0 (Unripe)/Test/IMG20200713145725.jpg"

# Preprocess the image
def preprocess_image(image_path, target_size=(224, 224)):
    # Load the image
    image = cv2.imread(image_path)
    # Resize to MobileNetV2 input size
    image_resized = cv2.resize(image, target_size)
    # Normalize to [0, 1]
    image_normalized = image_resized / 255.0
    # Add batch dimension
    image_expanded = np.expand_dims(image_normalized, axis=0)
    return image_expanded

# Preprocess the input image
input_image = preprocess_image(image_path)


In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
import os
import numpy as np
import matplotlib.pyplot as plt


In [8]:
base_dir = '/kaggle/input/ripeness-detection-of-mango'

# Define image dimensions and batch size
IMG_SIZE = (224, 224)  # Required input size for MobileNetV2
BATCH_SIZE = 32


In [ ]:
# Training data generator
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize pixel values
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 80-20 split for train-validation
)

# Training data loader
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

# Validation data loader
val_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)


In [ ]:
# Load MobileNetV2 without the top layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers
base_model.trainable = False

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output_layer = Dense(train_generator.num_classes, activation='softmax')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Display model summary
model.summary()


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=25,  # Start with fewer epochs to test
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_steps=val_generator.samples // BATCH_SIZE
)


In [ ]:
# Evaluate on validation data
val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")

# Save the model
model.save('mango_ripeness_mobilenetv2.h5')


In [ ]:
# Load the saved model
model = tf.keras.models.load_model('mango_ripeness_mobilenetv2.h5')

# Test with a single image
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Path to the test image
test_image_path = '/kaggle/input/ripeness-detection-of-mango/Stage_0 (Unripe)/Test/IMG20200713142258.jpg'

# Preprocess the test image
image = load_img(test_image_path, target_size=IMG_SIZE)
image_array = img_to_array(image) / 255.0
image_array = np.expand_dims(image_array, axis=0)

# Predict ripeness stage
prediction = model.predict(image_array)
predicted_class = np.argmax(prediction)
class_labels = list(train_generator.class_indices.keys())

print(f"Predicted Class: {class_labels[predicted_class]}")


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('mango_ripeness_mobilenetv2.h5')
tflite_model = converter.convert()
with open('mango_ripeness_model.tflite', 'wb') as f:
    f.write(tflite_model)
